In [ ]:
from time import perf_counter as pc
import warnings

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

from dpilqr import split_agents, plot_solve, split_agents_gen
import dpilqr as dec
import pocketknives

π = np.pi
g = 9.80665

In [ ]:
import util

In [ ]:
%load_ext autoreload
%autoreload 1

In [ ]:
def paper_setup_3_quads():
    x0 = np.array([[0.5, 1.5, 1, 0.5, 0.0, 0,
                    2.5, 1.5, 1, -0.5, 0.0, 0,
                    1.5, 1.3, 1, 0, 0.2, 0]], 
                     dtype=float).T
    xf = np.array([[2.5, 1.5, 1, 0, 0, 0, 
                    0.5, 1.5, 1, 0, 0, 0, 
                    1.5, 2.2, 1, 0, 0, 0]]).T
    # x0[dec.pos_mask([6]*3, 3)] += 0.1*np.random.randn(9, 1)
    # xf[dec.pos_mask([6]*3, 3)] += 0.1*np.random.randn(9, 1)
    return x0, xf

## 3 Drones

In [ ]:
n_agents = 3
n_states = 6
n_controls = 3

x_dims = [n_states] * n_agents
u_dims = [n_controls] * n_agents

n_dims = [3,3,3]

dt = 0.05

radius = 0.5

x0, xf = paper_setup_3_quads()

Q = np.diag([1, 1, 1, 5, 5, 5])
R = np.diag([0, 1, 1])
Qf = 1e3 * np.eye(n_states)

Qs = [Q] * n_agents
Rs = [R] * n_agents
Qfs = [Qf] * n_agents


ids = [100 + i for i in range(n_agents)]
model = dec.QuadcopterDynamics6D
dynamics = dec.MultiDynamicalModel([model(dt, id_) for id_ in ids])


goal_costs = [
    dec.ReferenceCost(xf_i, Q.copy(), R.copy(), Qf.copy(), id_)
    for xf_i, id_, x_dim, u_dim in zip(
        dec.split_agents_gen(xf, x_dims), ids, x_dims, u_dims
    )
]
prox_cost = dec.ProximityCost(x_dims, radius, n_dims)
goal_costs = [dec.ReferenceCost(xf_i, Q.copy(), R.copy(), Qf.copy(), id_) 
              for xf_i, id_ in zip(split_agents(xf.T, x_dims), ids)]
game_cost = dec.GameCost(goal_costs, prox_cost)

problem = dec.ilqrProblem(dynamics, game_cost)
# solver = dec.ilqrSolver(problem, N)


In [ ]:
problem.dynamics.n_u

In [ ]:
dec.solve_rhc?

In [ ]:
%%time
#Decentralized without multiprocessing:
step_size = 1
n_d = 3
N = 15
# U0 = np.zeros((N, dynamics.n_u))
X_trj, U, J = dec.solve_rhc(
    problem, x0, N, radius,
    centralized=False,
    n_d=n_d,
    step_size=step_size, 
    dist_converge=0.1,
    verbose=True,
    t_kill=step_size*dt,
    t_diverge=None
)

In [ ]:
%matplotlib ipympl
plt.figure(dpi=150)
plot_solve(X_trj, J, xf, x_dims, n_d=3)
plt.savefig('results/3_drones_DP_ILQR.png')

In [ ]:
!pip install imagemagick

In [ ]:
import matplotlib.animation as animation


# first, fill X_trj with some test data


# second, create a function that updates the scatter plot for each frame
def update_plot(k, X_trj, scatters):
     # Set the data for each scatter plot
    scatters[0]._offsets3d = X_trj.T[0:3, :k]
    scatters[1]._offsets3d = X_trj.T[6:9, :k]
    scatters[2]._offsets3d = X_trj.T[12:15, :k]
    return scatters

# Create the figure and axis
fig = plt.figure()
ax = plt.axes(projection='3d')

# Create the scatter plots
scatters = []
scatters.append(ax.scatter([], [], []))
scatters.append(ax.scatter([], [], []))
scatters.append(ax.scatter([], [], []))

# set the axis limits
ax.set_xlim3d(X_trj.T[[0, 6, 12], :].min(), X_trj.T[[0, 6, 12], :].max())
ax.set_ylim3d(X_trj.T[[1, 7, 13], :].min(), X_trj.T[[1, 7, 13], :].max())
ax.set_zlim3d(X_trj.T[[2, 8, 14], :].min(), X_trj.T[[2, 8, 14], :].max())


# Set the title
# ax.set_title('Trajectory from one-shot optimization (human + drones)')

ani = animation.FuncAnimation(fig, update_plot, frames=500, fargs=(X_trj, scatters))

ani.save('animation(3_drones_ILQR).gif',dpi = 150)
plt.show()

## 5 Drones

In [ ]:
n_agents = 5
n_states = 6
n_controls = 3

x_dims = [n_states] * n_agents
u_dims = [n_controls] * n_agents

n_dims = [3,3,3,3,3]

dt = 0.05

radius = 0.6

x0, xf = util.paper_setup_5_quads()

Q = np.diag([1, 1, 1, 5, 5, 5])
R = np.diag([0, 1, 1])
Qf = 1e3 * np.eye(n_states)

Qs = [Q] * n_agents
Rs = [R] * n_agents
Qfs = [Qf] * n_agents


ids = [100 + i for i in range(n_agents)]
model = dec.QuadcopterDynamics6D
dynamics = dec.MultiDynamicalModel([model(dt, id_) for id_ in ids])


goal_costs = [
    dec.ReferenceCost(xf_i, Q.copy(), R.copy(), Qf.copy(), id_)
    for xf_i, id_, x_dim, u_dim in zip(
        dec.split_agents_gen(xf, x_dims), ids, x_dims, u_dims
    )
]
prox_cost = dec.ProximityCost(x_dims, radius, n_dims)
goal_costs = [dec.ReferenceCost(xf_i, Q.copy(), R.copy(), Qf.copy(), id_) 
              for xf_i, id_ in zip(split_agents(xf.T, x_dims), ids)]
game_cost = dec.GameCost(goal_costs, prox_cost)

problem = dec.ilqrProblem(dynamics, game_cost)
# solver = dec.ilqrSolver(problem, N)


In [ ]:
%%time
#Decentralized without multiprocessing:
step_size = 1
n_d = 3
N = 15
# U0 = np.zeros((N, dynamics.n_u))
X_trj, U, J = dec.solve_rhc(
    problem, x0, N, radius,
    centralized=False,
    n_d=n_d,
    step_size=step_size, 
    dist_converge=0.1,
    verbose=True,
    t_kill=step_size*dt,
    t_diverge=None
)

In [ ]:
%matplotlib ipympl
plt.figure(dpi=150)
plot_solve(X_trj, J, xf, x_dims, n_d=3)

plt.savefig('results/5_drones_DP_ILQR.png')

In [ ]:
import matplotlib.animation as animation

# second, create a function that updates the scatter plot for each frame
def update_plot(k, X_trj, scatters):
     # Set the data for each scatter plot
    scatters[0]._offsets3d = X_trj.T[0:3, :k]
    scatters[1]._offsets3d = X_trj.T[6:9, :k]
    scatters[2]._offsets3d = X_trj.T[12:15, :k]
    scatters[3]._offsets3d = X_trj.T[18:21, :k]
    scatters[4]._offsets3d = X_trj.T[24:27, :k]
    return scatters


In [ ]:
# Create the figure and axis
fig = plt.figure()
ax = plt.axes(projection='3d')

# Create the scatter plots
scatters = []
scatters.append(ax.scatter([], [], []))
scatters.append(ax.scatter([], [], []))
scatters.append(ax.scatter([], [], []))
scatters.append(ax.scatter([], [], []))
scatters.append(ax.scatter([], [], []))


# set the axis limits
ax.set_xlim3d(X_trj.T[[0, 6, 12, 18, 24], :].min(), X_trj.T[[0, 6, 12, 18, 24], :].max())
ax.set_ylim3d(X_trj.T[[1, 7, 13, 19, 25], :].min(), X_trj.T[[1, 7, 13, 19, 25], :].max())
ax.set_zlim3d(X_trj.T[[2, 8, 14, 20, 26], :].min(), X_trj.T[[2, 8, 14, 20, 26], :].max())


# Set the title
# ax.set_title('Trajectory from one-shot optimization (human + drones)')

ani = animation.FuncAnimation(fig, update_plot, frames=500, fargs=(X_trj, scatters))

ani.save('animation(5_drones_ILQR).gif',dpi = 150)
plt.show()